## Preprocess

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import re

In [ ]:
original_train_sentences = 
original_labels_train = 

original_test_sentences = 
original_labels_test = 

original_sentences = original_train_sentences + original_test_sentences
labels = original_labels_train + original_labels_test

train_size = len(original_train_sentences)
test_size = len(original_test_sentences)

### Convert labels into one-hot

In [ ]:
num_class = len(list(set(labels)))

from sklearn.preprocessing import LabelEncoder
lEnc = LabelEncoder()
lEnc.fit(np.unique(labels))

num_labels_train = lEnc.transform(original_labels_train)
num_labels_test = lEnc.transform(original_labels_test)


train_labels = []
test_labels = []

for l in num_labels_train:
    to_add = [0]*num_class
    to_add[l]=1
    train_labels.append(to_add)

for l in num_labels_test:
    to_add = [0]*num_class
    to_add[l]=1
    test_labels.append(to_add)

### Remove Stopwords and less frequent words, tokenize sentences

In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
import sys


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'who', 'how', 'have', 'being', "mustn't", 'whom', 'as', "couldn't", 'did', 'few', 'wasn', 'both', 'between', 're', 'their', 'an', 'she', "she's", 'that', "needn't", 'if', "you've", 'the', 'own', 'won', "haven't", 'too', 'same', "weren't", 'nor', 'her', 't', "hadn't", 'itself', 'do', 'each', 'to', 'my', 'd', 'been', 'hadn', 've', 'its', 'themselves', 'just', 'it', 'this', 'has', 'hasn', 'they', 'doing', 'before', 'll', 'having', 'until', 'there', 'myself', 'no', 'other', 'hers', 'off', "that'll", 'when', 'ma', 'aren', 'our', 'doesn', 'wouldn', "won't", 'was', 'mightn', 'of', 'his', 'can', 'once', 'only', 'o', 'up', 'with', "hasn't", 'on', 'in', 'from', 'where', "you'll", 'shan', 'very', 'after', 'theirs', 'out', 'shouldn', 'be', 'through', 'will', 'didn', 'we', "wasn't", 'here', "aren't", 'during', 'ourselves', 'further', 'ain', 'more', 'or', 'don', 'now', 'is', 'does', 'had',

In [ ]:
def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


In [ ]:
original_word_freq = {}  # to remove rare words
for sentence in original_sentences:
    temp = clean_str(sentence)
    word_list = temp.split()
    for word in word_list:
        if word in original_word_freq:
            original_word_freq[word] += 1
        else:
            original_word_freq[word] = 1

In [ ]:
tokenize_sentences = []
word_list_dict = {}
for sentence in original_sentences:
    temp = clean_str(sentence)
    word_list_temp = temp.split()
    doc_words = []
    for word in word_list_temp:
        if word not in stop_words and original_word_freq[word] >= 5:
            doc_words.append(word)
            word_list_dict[word] = 1
    tokenize_sentences.append(doc_words)
word_list = list(word_list_dict.keys())
vocab_length = len(word_list)

del original_sentences

In [ ]:
#word to id dict
word_id_map = {}
for i in range(vocab_length):
    word_id_map[word_list[i]] = i

## Build Graph

In [ ]:
import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.spatial.distance import cosine

### PMI

In [ ]:
window_size = 20

In [ ]:
# word co-occurence with context windows
import collections
def get_dict(list_words):
    out_dict ={}
    unique_words = list(set(list_words))
    for i in range(1, len(unique_words)):
        word1 = unique_words[i]
        for j in range(0,i):
            word2 =  unique_words[j]
            word_id1 = word_id_map[word1]
            word_id2 = word_id_map[word2]
            out_dict[str(word_id1)+','+str(word_id2)]=1
            out_dict[str(word_id2)+','+str(word_id1)]=1
    return out_dict

windows=[]
progress = 0.0
sentence_list_of_dict=[]
for words in tokenize_sentences:
    list_of_dict = []
    step = int(len(tokenize_sentences)/25)
    progress+=1
    if(progress%step==0):
        print(str(int(progress/len(tokenize_sentences)*100))+"%")
    length = len(words)
    if length <= window_size:
        unique_words = list(set(words))
        windows.append(unique_words)
        list_of_dict.append(get_dict(words))
    else:
        for j in range(length - window_size + 1):           
            window = words[j: j + window_size]
            windows.append(list(set(window)))
            if(j==0):
                last_round_dict = get_dict(window)
            else:
                this_round_dict = last_round_dict.copy()
                new_word = window[-1]
                delete_word = words[j-1]
                if(new_word!=delete_word):
                    if(new_word not in window[:-1]):
                        for this_word in list(set(window[:-1])):
                            word1_id = word_id_map[this_word]
                            word2_id = word_id_map[new_word]
                            this_round_dict[str(word1_id)+','+str(word2_id)]=1
                            this_round_dict[str(word2_id)+','+str(word1_id)]=1
                    if(delete_word not in window[:-1]):
                        for this_word in list(set(window[:-1])):
                            word1_id = word_id_map[this_word]
                            word2_id = word_id_map[delete_word]
                            del this_round_dict[str(word1_id)+','+str(word2_id)]
                            del this_round_dict[str(word2_id)+','+str(word1_id)]
                last_round_dict = this_round_dict.copy()
            list_of_dict.append(last_round_dict)
    counter = collections.Counter() 
    for d in list_of_dict:  
        counter.update(d)       
    word_pair_count = dict(counter) 
    sentence_list_of_dict.append(word_pair_count)

3%
7%
11%
15%
19%
23%
27%
31%
35%
39%
43%
47%
51%
55%
59%
63%
67%
71%
75%
79%
83%
87%
91%
95%
99%


In [ ]:
import collections

counter = collections.Counter() 
for d in sentence_list_of_dict:  
    counter.update(d) 
      
word_pair_count = dict(counter) 

del sentence_list_of_dict

In [ ]:
word_window_freq = {}
for window in windows:
    for i in range(len(window)):
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1

In [ ]:
row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[word_list[i]]
    word_freq_j = word_window_freq[word_list[j]]
    pmi = log((count * num_window) /(word_freq_i * word_freq_j))
    if pmi <= 0:
        continue
    row.append(train_size + i)
    col.append(train_size + j)
    weight.append(pmi)

del word_window_freq
del word_pair_count

### Tf-idf

In [ ]:
#get each word appears in which document
word_doc_list = {}
for word in word_list:
    word_doc_list[word]=[]

for i in range(len(tokenize_sentences)):
    doc_words = tokenize_sentences[i]
    unique_words = set(doc_words)
    for word in unique_words:
        exsit_list = word_doc_list[word]
        exsit_list.append(i)
        word_doc_list[word] = exsit_list

#document frequency
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)

# term frequency
doc_word_freq = {}

for doc_id in range(len(tokenize_sentences)):
    words = tokenize_sentences[doc_id]
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1

In [ ]:
for i in range(len(tokenize_sentences)):
    words = tokenize_sentences[i]
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size:
            row.append(i)
        else:
            row.append(i + vocab_length)
        col.append(train_size + j)
        idf = log(1.0 * len(tokenize_sentences) / word_doc_freq[word_list[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)

del word_doc_freq
del doc_word_freq

### Adjacent matrix

In [ ]:
node_size = train_size + vocab_length + test_size
adj = sp.csr_matrix((weight, (row, col)), shape=(node_size, node_size))

## Build Model

In [ ]:
# from inits import *
import tensorflow as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res


class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])



class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)],
                              sparse=self.sparse_inputs)
            else:
                pre_sup = self.vars['weights_' + str(i)]            
            support = dot(self.support[i], pre_sup, sparse=True)
            supports.append(support)
        output = tf.add_n(supports)

        # bias
        if self.bias:
            output += self.vars['bias']
			
		
        self.embedding = output #output
        return self.act(output)


In [ ]:
# from metrics import *
import tensorflow as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)


class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])
        self.pred = tf.argmax(self.outputs, 1)
        self.labels = tf.argmax(self.placeholders['labels'], 1)

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            featureless=True,
                                            sparse_inputs=True,
                                            logging=self.logging))
        
        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x, #
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [ ]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    print(preds)
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))

    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf

from sklearn import metrics
import random
import os
import sys


# # Set random seed
# seed = random.randint(1, 200)
# np.random.seed(seed)
# tf.set_random_seed(seed)


# Settings
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

flags = tf.app.flags
FLAGS = flags.FLAGS
# 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('dataset', dataset, 'Dataset string.')
# 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_string('model', 'gcn', 'Model string.')
flags.DEFINE_float('learning_rate', 0.02, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 200, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 0,
                   'Weight for L2 loss on embedding matrix.')  # 5e-4
flags.DEFINE_integer('early_stopping', 100,
                     'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

### Build symetric adjacent matrix

In [ ]:
 adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

### Construct features (for original GCN is identity matrix)

In [ ]:
import scipy.sparse as sp
features = sp.identity(adj.shape[0])

def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape
    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)
    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)

features = preprocess_features(features)

### Construct y and mask

In [ ]:
real_train_size = int(0.9*train_size)
val_size = train_size-real_train_size

y_train=np.array([[0]*num_class]*adj.shape[0])
y_val = np.array([[0]*num_class]*adj.shape[0])
y_test=np.array([[0]*num_class]*adj.shape[0])

y_train[:real_train_size] = train_labels[:real_train_size]
y_val[real_train_size:train_size] = train_labels[real_train_size:]
y_test[-test_size:] = test_labels

train_mask = [False]*adj.shape[0]
val_mask = [False]*adj.shape[0]
test_mask = [False]*adj.shape[0]

train_mask[:real_train_size] = [True]*real_train_size
val_mask[real_train_size:train_size] = [True]*val_size
test_mask[-test_size:] = [True]*test_size

### Normalize adj

In [ ]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)

support = [preprocess_adj(adj)]

## Train the model

In [ ]:
num_supports = 1

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    # helper variable for sparse dropout
    'num_features_nonzero': tf.placeholder(tf.int32)
}

# Create model
print(features[2][1])
model = GCN(placeholders, input_dim=features[2][1], logging=True)

118685
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("graphconvolution_2/SparseTensorDenseMatMul/SparseTensorDenseMatMul:0", shape=(?, 4), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:


# Initialize session
session_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=session_conf)


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(
        features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], outs_val[2], outs_val[3], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []



In [ ]:
def construct_feed_dict(features, support, labels, labels_mask, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['support'][i]: support[i]
                      for i in range(len(support))})
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape})
    return feed_dict

In [ ]:
# Train model
for epoch in range(FLAGS.epochs):
    
    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(
        features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy,
                     model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict)

    # Validation
    cost, acc, pred, labels, duration = evaluate(
        features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(
              outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 1.38627 train_acc= 0.37127 val_loss= 1.37124 val_acc= 0.56989 time= 2.70464
Epoch: 0002 train_loss= 1.37095 train_acc= 0.57296 val_loss= 1.33902 val_acc= 0.56689 time= 2.13453
Epoch: 0003 train_loss= 1.33844 train_acc= 0.57057 val_loss= 1.29264 val_acc= 0.56700 time= 2.11171
Epoch: 0004 train_loss= 1.29052 train_acc= 0.56953 val_loss= 1.23491 val_acc= 0.56833 time= 2.12746
Epoch: 0005 train_loss= 1.23262 train_acc= 0.57199 val_loss= 1.17090 val_acc= 0.57155 time= 2.09992
Epoch: 0006 train_loss= 1.16925 train_acc= 0.57391 val_loss= 1.10827 val_acc= 0.57522 time= 2.10584
Epoch: 0007 train_loss= 1.10344 train_acc= 0.57807 val_loss= 1.05577 val_acc= 0.58000 time= 2.11155
Epoch: 0008 train_loss= 1.05183 train_acc= 0.58164 val_loss= 1.01883 val_acc= 0.58455 time= 2.11115
Epoch: 0009 train_loss= 1.01380 train_acc= 0.58674 val_loss= 0.99520 val_acc= 0.59211 time= 2.13291
Epoch: 0010 train_loss= 0.98915 train_acc= 0.59563 val_loss= 0.97613 val_acc= 0.60389 time= 2.12459


### Test the model

In [ ]:
# Testing
test_cost, test_acc, pred, labels, test_duration = evaluate(
    features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

test_pred = []
test_labels = []
print(len(test_mask))
for i in range(len(test_mask)):
    if test_mask[i]:
        test_pred.append(pred[i])
        test_labels.append(labels[i])

print("Test Precision, Recall and F1-Score...")
print(metrics.classification_report(test_labels, test_pred, digits=4))
print("Macro average Test Precision, Recall and F1-Score...")
print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='macro'))
print("Micro average Test Precision, Recall and F1-Score...")
print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='micro'))

# doc and word embeddings
print('embeddings:')
word_embeddings = outs[3][train_size: adj.shape[0] - test_size]
train_doc_embeddings = outs[3][:train_size]  # include val docs
test_doc_embeddings = outs[3][adj.shape[0] - test_size:]

Test set results: cost= 0.67268 accuracy= 0.76690 time= 0.72964
118685
Test Precision, Recall and F1-Score...
              precision    recall  f1-score   support

           0     0.8388    0.8256    0.8321      2735
           1     0.4882    0.3120    0.3807       532
           2     0.7946    0.8724    0.8317      5407
           3     0.5116    0.3982    0.4478      1326

    accuracy                         0.7669     10000
   macro avg     0.6583    0.6021    0.6231     10000
weighted avg     0.7529    0.7669    0.7569     10000

Macro average Test Precision, Recall and F1-Score...
(0.6583219083185988, 0.6020504789974914, 0.6231022711983202, None)
Micro average Test Precision, Recall and F1-Score...
(0.7669, 0.7669, 0.7669, None)
embeddings:


## Upload Word embeddings from original GCN

In [ ]:
word_embeddings = outs[3][train_size: adj.shape[0] - test_size]

word_emb_dict={}
for i in range(len(word_list)):
  word_emb_dict[word_list[i].strip()] = word_embeddings[i].tolist()

In [ ]:
import json

json.dump( word_emb_dict, open( "GCN_emb.json", 'w' ) )